In [ ]:
import gensim
import sqlite3
from operator import itemgetter

In [ ]:
class WordSplitter(object):
    def __init__(self, filename):
        self.filename = filename
 
    def __iter__(self):
        with open(self.filename) as fin:
            for line in fin:
                yield line.split()

model = gensim.models.Word2Vec(model_input, min_count=4)

In [ ]:
model.save(open('zoo/15/songs.word2vec', 'wb'))

In [ ]:
conn = sqlite3.connect('data/songs.db')
def find_song(song_name, limit=10):
    c = conn.cursor()
    c.execute("SELECT * FROM songs WHERE UPPER(name) LIKE '%" + song_name + "%'")
    res = sorted((x + (model.wv.vocab[x[0]].count,) 
                  for x in c.fetchall() if x[0] in model.wv.vocab),
                 key=itemgetter(-1), reverse=True)
    return [*res][:limit]

for t in find_song('the eye of the tiger'):
    print(*t)

In [ ]:
def suggest_songs(song_id):
    c = conn.cursor()
    similar = dict(model.most_similar([song_id]))
    song_ids = ', '.join(("'%s'" % x) for x in similar.keys())
    c.execute("SELECT * FROM songs WHERE id in (%s)" % song_ids)
    res = sorted((rec + (similar[rec[0]],) for rec in c.fetchall()), 
                 key=itemgetter(-1),
                 reverse=True)
    return [*res]


for t in suggest_songs('4rr0ol3zvLiEBmep7HaHtx'):
    print(*t)

In [ ]:
x = model.wv.vocab['2yqVlvXgZ51ynGqwfYADx4']
x.count